In [1]:
# Библиотека для парсинга HTML
!pip install beautifulsoup4

In [2]:
# Библиотека для инференса модели BAAI/bge-m3 (би-энкодера)
!pip install -U FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.5 MB/s eta 0:0

In [3]:
# Библиотека векторной базы данных для хранения эмбедингов и их быстрого поиска
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 18.6 MB/s eta 0:00:00


In [4]:
# Импортируем библиотеки
from bs4 import BeautifulSoup
import requests
import os
import zipfile
from tqdm.notebook import tqdm

import torch.nn.functional as F
import pandas as pd

from torch import Tensor
from transformers import AutoTokenizer, AutoModel

from FlagEmbedding import BGEM3FlagModel
import numpy as np
import faiss

import json
import requests

import re

Предварительно очищаем HTML от дополнительных теггов, которые были проставлены серисом DEDOC (удаление sub).

In [5]:
def clean_sub(html_doc, html_doc_clean):
    with open (html_doc) as f:
        html = f.read()
    # Создаем объект BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Находим все теги <sub> и удаляем их
    for tag in soup.find_all('sub'):
        tag.decompose()

    # Выводим результат
    html_out = soup.prettify()
    with open (html_doc_clean, "wt") as f:
        f.write (html_out)

In [6]:
# Проверка
# clean_sub("/content/page_10.html", "/content/page_10_clean.html")

In [7]:
# Функция очистки HTML файлов в пакетном режиме
def main_clean_html(path_in):
    # Папки с каталогами
    catalog_folders = [f"document_{i+1}_html" for i in range(7)]

    for folder in catalog_folders:
        folder = os.path.join(path_in, folder)
        if not os.path.exists(folder):
            print(f"Папка {folder} не найдена. Пропускаем.")
            continue

        # Создаем папку для очищенных HTML-файлов
        html_folder = os.path.join(folder, "clean_html")
        os.makedirs(html_folder, exist_ok=True)
        print(f"Обрабатываю папку: {folder}")

        # Обрабатываем каждый html на гугл диске
        for html_file in tqdm(os.listdir(folder)):
            if not html_file.endswith(".html"):
                continue

            html_path = os.path.join(folder, html_file)
            html_clean_path = os.path.join(html_folder, html_file)

            # Очищаем HTML
            clean_sub(html_path, html_clean_path)
            print(f"Файл {html_clean_path} очищен из {html_file}.")

        # Архивируем папку с HTML-файлами
        zip_path = f"{folder}_html.zip"
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            for root, _, files_in_folder in os.walk(html_folder):
                for file in files_in_folder:
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, arcname=os.path.relpath(file_path, html_folder))

        print(f"Папка {folder} с HTML-файлами архивирована в {zip_path}.")

main_clean_html("/content/drive/MyDrive/006 html")

Обрабатываю папку: /content/drive/MyDrive/006 html/document_1_html


  0%|          | 0/329 [00:00<?, ?it/s]

Файл /content/drive/MyDrive/006 html/document_1_html/clean_html/page_302.html очищен из page_302.html.
Файл /content/drive/MyDrive/006 html/document_1_html/clean_html/page_230.html очищен из page_230.html.
Файл /content/drive/MyDrive/006 html/document_1_html/clean_html/page_282.html очищен из page_282.html.
Файл /content/drive/MyDrive/006 html/document_1_html/clean_html/page_9.html очищен из page_9.html.
Файл /content/drive/MyDrive/006 html/document_1_html/clean_html/page_172.html очищен из page_172.html.
Файл /content/drive/MyDrive/006 html/document_1_html/clean_html/page_120.html очищен из page_120.html.
Файл /content/drive/MyDrive/006 html/document_1_html/clean_html/page_272.html очищен из page_272.html.
Файл /content/drive/MyDrive/006 html/document_1_html/clean_html/page_243.html очищен из page_243.html.
Файл /content/drive/MyDrive/006 html/document_1_html/clean_html/page_219.html очищен из page_219.html.
Файл /content/drive/MyDrive/006 html/document_1_html/clean_html/page_216.html

  0%|          | 0/244 [00:00<?, ?it/s]

Файл /content/drive/MyDrive/006 html/document_2_html/clean_html/page_211.html очищен из page_211.html.
Файл /content/drive/MyDrive/006 html/document_2_html/clean_html/page_86.html очищен из page_86.html.
Файл /content/drive/MyDrive/006 html/document_2_html/clean_html/page_243.html очищен из page_243.html.
Файл /content/drive/MyDrive/006 html/document_2_html/clean_html/page_216.html очищен из page_216.html.
Файл /content/drive/MyDrive/006 html/document_2_html/clean_html/page_53.html очищен из page_53.html.
Файл /content/drive/MyDrive/006 html/document_2_html/clean_html/page_172.html очищен из page_172.html.
Файл /content/drive/MyDrive/006 html/document_2_html/clean_html/page_219.html очищен из page_219.html.
Файл /content/drive/MyDrive/006 html/document_2_html/clean_html/page_70.html очищен из page_70.html.
Файл /content/drive/MyDrive/006 html/document_2_html/clean_html/page_227.html очищен из page_227.html.
Файл /content/drive/MyDrive/006 html/document_2_html/clean_html/page_88.html оч

...

Файл /content/drive/MyDrive/006 html/document_5_html/clean_html/page_10.html очищен из page_10.html.
Файл /content/drive/MyDrive/006 html/document_5_html/clean_html/page_5.html очищен из page_5.html.
Файл /content/drive/MyDrive/006 html/document_5_html/clean_html/page_8.html очищен из page_8.html.
Файл /content/drive/MyDrive/006 html/document_5_html/clean_html/page_3.html очищен из page_3.html.
Файл /content/drive/MyDrive/006 html/document_5_html/clean_html/page_9.html очищен из page_9.html.
Файл /content/drive/MyDrive/006 html/document_5_html/clean_html/page_7.html очищен из page_7.html.
Файл /content/drive/MyDrive/006 html/document_5_html/clean_html/page_2.html очищен из page_2.html.
Файл /content/drive/MyDrive/006 html/document_5_html/clean_html/page_1.html очищен из page_1.html.
Файл /content/drive/MyDrive/006 html/document_5_html/clean_html/page_6.html очищен из page_6.html.
Файл /content/drive/MyDrive/006 html/document_5_html/clean_html/page_4.html очищен из page_4.html.
Папка /c

  0%|          | 0/11 [00:00<?, ?it/s]

Файл /content/drive/MyDrive/006 html/document_6_html/clean_html/page_10.html очищен из page_10.html.
Файл /content/drive/MyDrive/006 html/document_6_html/clean_html/page_5.html очищен из page_5.html.
Файл /content/drive/MyDrive/006 html/document_6_html/clean_html/page_8.html очищен из page_8.html.
Файл /content/drive/MyDrive/006 html/document_6_html/clean_html/page_6.html очищен из page_6.html.
Файл /content/drive/MyDrive/006 html/document_6_html/clean_html/page_1.html очищен из page_1.html.
Файл /content/drive/MyDrive/006 html/document_6_html/clean_html/page_4.html очищен из page_4.html.
Файл /content/drive/MyDrive/006 html/document_6_html/clean_html/page_2.html очищен из page_2.html.
Файл /content/drive/MyDrive/006 html/document_6_html/clean_html/page_9.html очищен из page_9.html.
Файл /content/drive/MyDrive/006 html/document_6_html/clean_html/page_7.html очищен из page_7.html.
Файл /content/drive/MyDrive/006 html/document_6_html/clean_html/page_3.html очищен из page_3.html.
Папка /c

  0%|          | 0/35 [00:00<?, ?it/s]

Файл /content/drive/MyDrive/006 html/document_7_html/clean_html/page_9.html очищен из page_9.html.
Файл /content/drive/MyDrive/006 html/document_7_html/clean_html/page_2.html очищен из page_2.html.
Файл /content/drive/MyDrive/006 html/document_7_html/clean_html/page_24.html очищен из page_24.html.
Файл /content/drive/MyDrive/006 html/document_7_html/clean_html/page_14.html очищен из page_14.html.
Файл /content/drive/MyDrive/006 html/document_7_html/clean_html/page_13.html очищен из page_13.html.
Файл /content/drive/MyDrive/006 html/document_7_html/clean_html/page_6.html очищен из page_6.html.
Файл /content/drive/MyDrive/006 html/document_7_html/clean_html/page_12.html очищен из page_12.html.
Файл /content/drive/MyDrive/006 html/document_7_html/clean_html/page_25.html очищен из page_25.html.
Файл /content/drive/MyDrive/006 html/document_7_html/clean_html/page_29.html очищен из page_29.html.
Файл /content/drive/MyDrive/006 html/document_7_html/clean_html/page_1.html очищен из page_1.html

Необходимо проверить работу токенизатора модели BAAI/bge-m3 с полученными HTML документами для того, чтобы примерно расчитать число токенов на один документ.

In [5]:
# Загружаем токенизатор из hugging face
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-m3')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

In [6]:
# Функция для расчёта количества токенов в одной странице
def count_tokens(file_name):
    with open (file_name) as f:
        html = f.read()
    tokens = tokenizer.encode(html, padding = False, truncation = False, return_tensors = "pt") # pt - PyTorch
    return tokens.shape[1] # [0] - объекты батча, [1] - токены

In [10]:
count_tokens("/content/page_1.html")

3010

Количество токенов оказалось значительно выше 512 (числа токенов для большинства модели эмбеддингов). Но модель **BAAI/bge-m3** поддерживает обработку до примерно 8000 токенов. Поэтому для этой модели нет необходимости дополнительной разбивки страниц на фрагменты.

In [7]:
# Загрузка модели BAAI/bge-m3 для расчёта эмбеддингов
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16 = True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

colbert_linear.pt:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

sparse_linear.pt:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

Функция для расчёта эмбеддинга для HTML

In [8]:
# Создание эмбеддингов для html файла
def create_embedding(file_name):
    with open (file_name) as f:
        html = f.read()
    embeddings = model.encode(html, batch_size = 1, max_length=8192)['dense_vecs']
    return embeddings

Проверим функцию на отдельно взятой странице.

In [13]:
embedding = create_embedding("/content/page_10.html")
embedding

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


array([-0.0414  ,  0.003183, -0.06015 , ...,  0.01098 ,  0.03366 ,
        0.00283 ], dtype=float16)

In [14]:
type(embedding)

numpy.ndarray

In [15]:
embedding.shape

(1024,)

Напишем функцию для создания таблицы (pandas) для заполнения базы данных документов (faiss):
- индекс - id
- 1 колонка - номер папки
- 2 колонка - имя страницы
- 3 колонка - эмбеддинг

In [16]:
def create_table(path_in):
    rows = []
    catalog_folders = [f"document_{i+1}_html" for i in range(7)]

    for folder in catalog_folders:
        folder = os.path.join(path_in, folder, "clean_html")
        if not os.path.exists(folder):
            print(f"Папка {folder} не найдена. Пропускаем.")
            continue

        # Обрабатываем каждый html на гугл диске
        for html_file in tqdm(os.listdir(folder)):
            if not html_file.endswith(".html"):
                continue

            html_path = os.path.join(folder, html_file)
            embedding = create_embedding(html_path)
            rows.append([folder, html_file, embedding])
    return pd.DataFrame(rows, columns = ["folder", "file_name", "embedding"])

In [17]:
dataframe = create_table("/content/drive/MyDrive/006 html")

  0%|          | 0/329 [00:00<?, ?it/s]

  0%|          | 0/244 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

In [19]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 669 entries, 0 to 668
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   folder     669 non-null    object
 1   file_name  669 non-null    object
 2   embedding  669 non-null    object
dtypes: object(3)
memory usage: 15.8+ KB


In [20]:
# Вытаскиваем номер каталога
dataframe["folder"] = dataframe["folder"].apply(lambda x: x.split("_")[1]).astype(int)
dataframe

,folder,file_name,embedding
0,1,page_302.html,"[-0.074, 0.03778, -0.04227, -0.02498, -0.00124..."
1,1,page_282.html,"[-0.0767, 0.03613, -0.05023, -0.01423, 0.00598..."
2,1,page_230.html,"[-0.05215, 0.03397, -0.06274, 0.006653, 0.0067..."
3,1,page_216.html,"[-0.02795, 0.01399, -0.04868, 0.03726, 0.00406..."
4,1,page_243.html,"[-0.039, 0.01258, -0.06018, 0.01714, 0.02441, ..."
...,...,...,...
664,7,page_22.html,"[-0.02077, 0.00924, -0.0227, 0.02536, -0.03555..."
665,7,page_18.html,"[-0.05658, -0.0009723, -0.03072, 0.01431, 0.01..."
666,7,page_10.html,"[-0.0538, -0.006287, -0.01575, 0.0499, -0.0158..."
667,7,page_19.html,"[-0.0537, 0.02495, -0.04333, 0.006714, 0.00483..."


In [21]:
# Вытаскиваем номер страницы
dataframe["file_name"] = dataframe["file_name"].apply(lambda x: x.split("_")[1]).apply(lambda y: y.split(".")[0]).astype(int)
dataframe

,folder,file_name,embedding
0,1,302,"[-0.074, 0.03778, -0.04227, -0.02498, -0.00124..."
1,1,282,"[-0.0767, 0.03613, -0.05023, -0.01423, 0.00598..."
2,1,230,"[-0.05215, 0.03397, -0.06274, 0.006653, 0.0067..."
3,1,216,"[-0.02795, 0.01399, -0.04868, 0.03726, 0.00406..."
4,1,243,"[-0.039, 0.01258, -0.06018, 0.01714, 0.02441, ..."
...,...,...,...
664,7,22,"[-0.02077, 0.00924, -0.0227, 0.02536, -0.03555..."
665,7,18,"[-0.05658, -0.0009723, -0.03072, 0.01431, 0.01..."
666,7,10,"[-0.0538, -0.006287, -0.01575, 0.0499, -0.0158..."
667,7,19,"[-0.0537, 0.02495, -0.04333, 0.006714, 0.00483..."


In [22]:
# Создаем матрицу эмбеддингов
matrix_embedding = np.row_stack(dataframe["embedding"].to_list())
matrix_embedding

<ipython-input-22-e4c0b193eed8>:2: DeprecationWarning: `row_stack` alias is deprecated. Use `np.vstack` directly.
  matrix_embedding = np.row_stack(dataframe["embedding"].to_list())


array([[-0.074   ,  0.03778 , -0.04227 , ...,  0.00648 ,  0.0342  ,
         0.02724 ],
       [-0.0767  ,  0.03613 , -0.05023 , ...,  0.01028 ,  0.04855 ,
         0.02422 ],
       [-0.05215 ,  0.03397 , -0.06274 , ...,  0.01878 ,  0.0642  ,
         0.01006 ],
       ...,
       [-0.0538  , -0.006287, -0.01575 , ..., -0.0267  , -0.0648  ,
        -0.03787 ],
       [-0.0537  ,  0.02495 , -0.04333 , ..., -0.009674, -0.0578  ,
        -0.01788 ],
       [-0.0736  ,  0.007687, -0.0329  , ..., -0.02425 ,  0.0697  ,
         0.002266]], dtype=float16)

In [23]:
# Проверяем вектора на евклидову норму
np.linalg.norm(matrix_embedding[0])

np.float16(0.9995)

In [24]:
# Проверяем размерность матрицы
matrix_embedding.shape

(669, 1024)

In [25]:
# Записываем в csv колонки с папкой и номером страницы
dataframe[["folder", "file_name"]].to_csv("catalog_page.csv")

In [26]:
dim = 1024 # размерность эмбеддинга

In [27]:
# Создаем faiss индекс
index = faiss.IndexFlatIP(dim) # IP - внутренний продукт (скалярное произведение)

In [28]:
# в индекс добавляем наши вектора
index.add(matrix_embedding)

In [29]:
# Проверяем количество веторов в индексе
index.ntotal

669

Проверяем работу индекса faiss с помощью запроса и поиску векторов документов семантически близких к запросу.

In [30]:
query = "Кто председатель совета директоров на заводе Энергокабель?"

In [31]:
query_embedding = model.encode(query, batch_size = 1, max_length = 512)['dense_vecs']

In [32]:
# D - косинусное сходство, I - индексы
D,I = index.search(query_embedding.reshape((1, 1024)), 10)

In [33]:
D,I

(array([[0.6158519 , 0.6152342 , 0.59250057, 0.5793488 , 0.5700127 ,
         0.5434154 , 0.5082164 , 0.50093687, 0.48982945, 0.48906785]],
       dtype=float32),
 array([[456, 260, 663, 542, 639, 644, 579, 647, 603, 600]]))

In [34]:
# сохраняем индекс в файл
faiss.write_index(index, "index.faiss")

In [35]:
# чтение индекса из файла
index = faiss.read_index("index.faiss")

In [36]:
dataframe_new = pd.read_csv("catalog_page.csv", index_col = 0)
dataframe_new

,folder,file_name
0,1,302
1,1,282
2,1,230
3,1,216
4,1,243
...,...,...
664,7,22
665,7,18
666,7,10
667,7,19


Напишем функцию для загрузки страниц (в строковом виде) по номерам найденных векторов.

In [37]:
# по индексам из faiss вытаскиваем сами страницы в строковом виде
def list_texts(df, list_index, path):
    selected_rows = df.iloc[list_index, :]
    print(selected_rows)
    list_text = []
    for index, row in selected_rows.iterrows():
        folder = row["folder"]
        folder_name = f"document_{folder}_html"
        page = row["file_name"]
        page_name = f"page_{page}.html"
        full_name = os.path.join(path, folder_name, "clean_html", page_name)
        with open (full_name) as f:
            html = f.read()
            list_text.append(html)
    return list_text

Протестируем функцию.

In [38]:
# список со всеми текстами, подходящими по запросу
list_texts(dataframe_new, [ 13, 107,   4,   8, 294,  14, 157, 142, 202, 161], "/content/drive/MyDrive/006 html")

     folder  file_name
13        1        217
107       1        218
4         1        243
8         1          9
294       1        113
14        1         53
157       1        199
142       1         99
202       1        222
161       1        122


['<p>\n <strong>\n </strong>\n</p>\n<p>\n стр. 226\n <br/>\n</p>\n<p>\n провода и кабели для электрических установок\n <br/>\n</p>\n<p>\n ОБЛАСТЬ ПРИМЕНЕНИЯ И НАЗНА ЧЕНИЕ\n <br/>\n</p>\n<p>\n Кабели предназначены для электрических установок при стацио-\n <br/>\n нарной прокладке в осветительных и силовых сетях, а также для\n <br/>\n монтажа электрооборудования, машин, механизмов и станков на\n <br/>\n номинальное переменное напряжение до 220/380 В частотой до\n <br/>\n 400 Г ц или постоянное напряжение до 700 В включительно.\n <br/>\n Кабели не распространяют горение при групповой стационарной\n <br/>\n прокладке кабельных линий и сохраняют работоспособность при\n <br/>\n воздействии пламени не менее 180 минут .\n <br/>\n</p>\n<p>\n Предназначены для электропроводок и цепей аварийного\n <br/>\n электроснабжения и питания оборудования (токоприемников)\n <br/>\n в жилых и общественных зданиях, в операционных от делениях\n <br/>\n больниц, в офисных помещениях, оснащенных компьютерной\n <

In [39]:
texts = list_texts(dataframe_new, I.flatten(), "/content/drive/MyDrive/006 html")

     folder  file_name
456       2        122
260       1          1
663       7         21
542       2          1
639       7         25
644       7          1
579       3          2
647       7         28
603       3         30
600       3         23


In [40]:
len(texts[2])

2339

Напишем функцию составления промта для большой языковой модели. Проверим эту функцию на найденных документах и поисковом запросе.

In [9]:
# функция для создания промта из текста запроса и текста документа
def generate_promt(query, list_texts):
    prefix = "Запрос на получение информации."
    prefix += f"\nЗапрос: {query}\n"
    for text in list_texts:
        prefix += f"Документ:\n{text}\n"
    prefix += "На основе представленных документов и запроса пользователя сформулируй ответ, учитывающий ключевые моменты и содержимое документов.\nУчти, что найденные документы и части могут не отвечать на запрос. Если список документов пустой, то ответь что по запросу ничего не найдено. Представь ответ в виде обычного текста."
    return prefix

In [42]:
generate_promt(query, texts)

'Запрос на получение информации.\nЗапрос: Кто председатель совета директоров на заводе Энергокабель?\nДокумент:\n<p>\n <strong>\n </strong>\n</p>\n<p>\n стр. 3\n <br/>\n</p>\n<p>\n СПРАВОЧНАЯ ИНФОРМАЦИЯ\n <br/>\n</p>\n<p>\n О ЗАВОДЕ\n <br/>\n</p>\n<p>\n Завод «Энергокабель» был создан в городе Электроугли Ногинского района Московской\n <br/>\n области по специальному проекту Г осударственного проектного института Министерства РФ\n <br/>\n по атомной энергии и Всероссийского НИИ кабельной промышленности. Первая продукция\n <br/>\n предприятия поступила на российский рынок в 2002 г . На протяжении всех лет работы завод\n <br/>\n интенсивно развивался, наращивал мощности, и в настоящее время его номенклатура включает\n <br/>\n десятки тысяч маркоразмеров продукции.\n <br/>\n</p>\n<p>\n Предприятие признано одним из лидеров отрасли по качеству выпускаемых изделий,\n <br/>\n и сегодня само название «Энергокабель» может служить синонимом надежности кабельно-\n <br/>\n проводниковой продукции

Напишем функцию для отправки запроса большой языковой модели, хостинг которой осуществлен на сервере VLLM. Формат запроса совместим с OPEN AI API.

In [71]:
# функция отправляет запрос по API и получает ответ
def send_request_to_vllm(api_url, prompt, model_name="default-model", max_tokens=150, temperature=0.4, headers=None, top_p=0.6,
                         length_penalty=1.6, repetition_penalty=1.1, no_repeat_ngram_size=4, do_sample=False, top_k=100, stream=False):
    """
    Отправляет POST-запрос в VLLM сервис для генерации текста.

    :param api_url: URL API сервиса VLLM
    :param prompt: Текст запроса (промпт) для модели
    :param model_name: Имя модели (опционально)
    :param max_tokens: Максимальное количество токенов в ответе (опционально)
    :param temperature: Параметр температуры для генерации (опционально)
    :param headers: Заголовки запроса (опционально)
    :return: Ответ от сервиса в формате JSON
    """
    # Подготовка данных для отправки
    payload = {"model": model_name,
            "messages": [
                {"role": "system", "content": "Ты — универсальный эксперт, готовый помочь с различными вопросами. Твоя задача — анализировать предоставленные документы их фрагменты и формировать точные и информативные ответы на вопросы пользователей."},
                {"role": "user", "content": prompt}
            ],
            "temperature": temperature,
            "top_p": top_p,
            "max_new_tokens": max_tokens,
            "length_penalty": length_penalty,
            "early_stopping": True,
            "repetition_penalty": repetition_penalty,
            "no_repeat_ngram_size": no_repeat_ngram_size,
            "do_sample": do_sample,
            "top_k": top_k,
            'stream': False  # Выключаем потоковую передачу
        }

    # Если заголовки не переданы, используем стандартные
    if headers is None:
        headers = {
            'Content-Type': 'application/json',
        }

    # Преобразуем данные в JSON
    json_data = json.dumps(payload)

    # Отправляем POST-запрос
    response = requests.post(api_url, data=json_data, headers=headers, timeout = 180)

    # Проверяем статус ответа
    if response.status_code == 200:
        # Возвращаем ответ
        return response.json()["choices"][0]["message"]["content"]
    else:
        raise ValueError("Ошибка запроса")

В качестве модели используем квантинизованную LLM Qwen2.5-3B-Instruct-GPTQ-Int8 .

In [12]:
# Параметры функции отправки запроса и получения ответа
api_url = 'http://158.255.5.135:1240/v1/chat/completions'
model_name = "Qwen/Qwen2.5-3B-Instruct-GPTQ-Int8"

In [ ]:
prompt = generate_promt(query, texts)

In [45]:
# сам запрос
response = send_request_to_vllm(api_url, prompt, model_name)
print(response)

По имеющимся данным, председателем Совета директоров АО "Завод 'Энергокабель'" является Дмитрий Викторович Пташинский.

Обратите внимание, что среди предоставленных документов нет конкретной записи о должности председателя Совета директоров. Все документы сосредоточены преимущественно на информационных данных о работе завода, его продукции и сертификационных удостоверениях. 

Отсутствие точной информации о руководстве указывает на возможность того, что данная информация могла быть изменена после последнего обновления документов, которые были рассмотрены. Для получения наиболее актуальной информации рекомендую обратиться напрямую к официальному сайту завода или связаться с ним через указанную контактную информацию.


Вариант телеграм бота, построенного с использованием HTML документов, распарсинных DEDOC показал довольно посредственные результаты в виду захламления контекста LLM лишними токенами (HTML теги и плохо распознанные данные). Попробуем выполнить очистку документов и перевод их в формат markdown с использованием LLM.

In [46]:
# Функция создания промпта для очистки и конвертации из формата HTML в формат markdown
def generate_prompt_2(text):
  prompt = "Переведи документ из html в markdown. Исправь орфографические ошибки.Не добавляй объяснение изменений!"
  prompt += text
  return prompt

Эксперименты с очисткой страниц.

In [47]:
with open("/content/drive/MyDrive/006 html/document_1_html/clean_html/page_1.html") as f:
    html = f.read()

In [48]:
# Параметры функции отправки запроса и получения ответа
prompt = generate_prompt_2(html)
response = send_request_to_vllm(api_url, prompt, model_name)
print(response)

# О ЗАВОДЕ

Завод «Энергокабель» был создан в городе Электроугли Ногинского района Московской области по специальному проекту  
Государственного проектного института Министерства РФ по атомной энергии и Всероссийского НИИ кабельной промышленности.

Первая продукция предприятия поступила на российский рынок в 2002 году. На протяжении всех лет работы завод интенсивно развивался, наращивал мощности, и в настоящее время его номенклатура включает десятки тысяч марочных размеров продукции.

Предприятие признано одним из лидеров отрасли по качеству выпускаемых изделий, и сегодня само название «Энергокабель» может служить синонимом надёжности кабельно-проводниковой продукции.

Марке завода доверяют как государственные ведомства, так и ведущие компании атомной, нефтегазовой, транспортной отраслей. Кабели подмосковного предприятия поставляются на объекты промышленного и гражданского строительства, а также специальных назначения по всей территории России.

В числе сильных сторон АО «Завод «Энерго

In [ ]:
# Функция для пакетной обработки с помощью LLM документов HTML в формат markdown
def main_markdown(path_in):
    # Папки с каталогами
    catalog_folders = [f"document_{i+1}_html" for i in range(7)]

    for folder in catalog_folders:
        folder = os.path.join(path_in, folder, "clean_html")
        if not os.path.exists(folder):
            print(f"Папка {folder} не найдена. Пропускаем.")
            continue

        # Создаем папку для файлов markdown
        markdown_folder = os.path.join(folder, "clean_markdown")
        os.makedirs(markdown_folder, exist_ok=True)
        print(f"Обрабатываю папку: {folder}")

        # Обрабатываем каждый html файл
        for html_file in tqdm(os.listdir(folder)):
            if not html_file.endswith(".html"):
                continue

            html_path = os.path.join(folder, html_file)
            markdown_path = os.path.join(markdown_folder, os.path.splitext(html_file)[0] + ".md")

            # Пропускаем, если markdown-файл уже существует
            if os.path.exists(markdown_path):
                print(f"Файл {html_file} уже обработан. Пропускаем.")
                continue

            try:
                # Чтение HTML-файла
                with open(html_path, "r", encoding="utf-8") as f:
                    html = f.read()

                # Генерация запроса
                prompt = generate_prompt_2(html)

                # Отправка запроса и получение ответа
                response = send_request_to_vllm(api_url, prompt, model_name)
                response = response.rstrip().rsplit("\n", 1)[0]

                # Запись результата в markdown-файл
                with open(markdown_path, "w", encoding="utf-8") as w:
                    w.write(response)

                print(f"Обработан файл: {html_file}")

            except Exception as e:
                print(f"Ошибка при обработке файла {html_file}: {e}")

main_markdown("/content/drive/MyDrive/006 html")

Обрабатываю папку: /content/drive/MyDrive/006 html/document_1_html/clean_html


  0%|          | 0/329 [00:00<?, ?it/s]

Обработан файл: page_302.html
Обработан файл: page_282.html
Обработан файл: page_230.html
Обработан файл: page_9.html
Обработан файл: page_120.html
Обработан файл: page_172.html
Обработан файл: page_243.html
Ошибка при обработке файла page_272.html: HTTPConnectionPool(host='158.255.5.135', port=1240): Read timed out. (read timeout=180)
Обработан файл: page_219.html
Обработан файл: page_216.html
Обработан файл: page_70.html
Ошибка при обработке файла page_86.html: HTTPConnectionPool(host='158.255.5.135', port=1240): Read timed out. (read timeout=180)
Обработан файл: page_210.html
Обработан файл: page_217.html
Обработан файл: page_53.html
Обработан файл: page_215.html
Обработан файл: page_293.html
Обработан файл: page_180.html
Обработан файл: page_241.html
Обработан файл: page_211.html
Обработан файл: page_227.html
Ошибка при обработке файла page_267.html: HTTPConnectionPool(host='158.255.5.135', port=1240): Read timed out. (read timeout=180)
Обработан файл: page_154.html
Обработан файл:

  0%|          | 0/244 [00:00<?, ?it/s]

Обработан файл: page_211.html
Обработан файл: page_86.html
Обработан файл: page_88.html
Обработан файл: page_172.html
Обработан файл: page_216.html
Обработан файл: page_243.html
Обработан файл: page_219.html
Обработан файл: page_227.html
Обработан файл: page_53.html
Обработан файл: page_70.html
Обработан файл: page_190.html
Обработан файл: page_120.html
Обработан файл: page_6.html
Обработан файл: page_217.html
Обработан файл: page_41.html
Обработан файл: page_192.html
Обработан файл: page_106.html
Обработан файл: page_154.html
Обработан файл: page_209.html
Обработан файл: page_72.html
Обработан файл: page_158.html
Обработан файл: page_13.html
Обработан файл: page_180.html
Обработан файл: page_37.html
Обработан файл: page_48.html
Обработан файл: page_103.html
Обработан файл: page_185.html
Обработан файл: page_159.html
Обработан файл: page_215.html
Обработан файл: page_117.html
Обработан файл: page_10.html
Обработан файл: page_2.html
Ошибка при обработке файла page_241.html: HTTPConnecti

...

Обработан файл: page_10.html
Обработан файл: page_5.html
Обработан файл: page_8.html
Обработан файл: page_3.html
Обработан файл: page_9.html
Обработан файл: page_7.html
Обработан файл: page_2.html
Обработан файл: page_1.html
Обработан файл: page_6.html
Обработан файл: page_4.html
Обрабатываю папку: /content/drive/MyDrive/006 html/document_6_html/clean_html


  0%|          | 0/11 [00:00<?, ?it/s]

Ошибка при обработке файла page_10.html: HTTPConnectionPool(host='158.255.5.135', port=1240): Read timed out. (read timeout=180)
Обработан файл: page_5.html
Обработан файл: page_6.html
Обработан файл: page_8.html
Обработан файл: page_4.html
Обработан файл: page_1.html
Обработан файл: page_2.html
Обработан файл: page_9.html
Обработан файл: page_7.html
Обработан файл: page_3.html
Обрабатываю папку: /content/drive/MyDrive/006 html/document_7_html/clean_html


  0%|          | 0/35 [00:00<?, ?it/s]

Обработан файл: page_9.html
Обработан файл: page_24.html
Обработан файл: page_2.html
Обработан файл: page_14.html
Обработан файл: page_13.html
Обработан файл: page_6.html
Обработан файл: page_12.html
Обработан файл: page_25.html
Обработан файл: page_29.html
Обработан файл: page_1.html
Обработан файл: page_33.html
Обработан файл: page_28.html
Обработан файл: page_26.html
Обработан файл: page_31.html
Обработан файл: page_32.html
Обработан файл: page_23.html
Обработан файл: page_27.html
Обработан файл: page_34.html
Обработан файл: page_30.html
Обработан файл: page_20.html
Обработан файл: page_5.html
Обработан файл: page_7.html
Обработан файл: page_16.html
Обработан файл: page_11.html
Обработан файл: page_15.html
Обработан файл: page_17.html
Обработан файл: page_8.html
Обработан файл: page_21.html
Обработан файл: page_3.html
Обработан файл: page_22.html
Обработан файл: page_10.html
Обработан файл: page_18.html
Обработан файл: page_19.html
Обработан файл: page_4.html


In [13]:
count_tokens("/content/page_1.md")
# Число токенов значительно уменьшилось

966

In [14]:
# Создаем функцию для создания таблицы под faiss
def create_table(path_in):
    rows = []
    catalog_folders = [f"document_{i+1}_html" for i in range(7)]

    for folder in catalog_folders:
        folder = os.path.join(path_in, folder, "clean_html", "clean_markdown")
        if not os.path.exists(folder):
            print(f"Папка {folder} не найдена. Пропускаем.")
            continue

        # Обрабатываем каждый markdown файл на гугл диске
        for md_file in tqdm(os.listdir(folder)):
            if not md_file.endswith(".md"):
                continue

            md_path = os.path.join(folder, md_file)
            embedding = create_embedding(md_path)
            rows.append([folder, md_file, embedding])
    return pd.DataFrame(rows, columns = ["folder", "file_name", "embedding"])

In [15]:
dataframe = create_table("/content/drive/MyDrive/006 html")

  0%|          | 0/291 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/228 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

In [16]:
dataframe.head()

,folder,file_name,embedding
0,/content/drive/MyDrive/006 html/document_1_htm...,page_86.md,"[-0.03644, 0.004272, -0.0418, -0.007412, 0.016..."
1,/content/drive/MyDrive/006 html/document_1_htm...,page_272.md,"[-0.02133, 0.01595, -0.055, -0.03708, -0.00193..."
2,/content/drive/MyDrive/006 html/document_1_htm...,page_288.md,"[-0.002651, 0.01232, -0.04044, -0.05136, 0.007..."
3,/content/drive/MyDrive/006 html/document_1_htm...,page_37.md,"[-0.02913, 0.02794, -0.0411, -0.03366, 0.01168..."
4,/content/drive/MyDrive/006 html/document_1_htm...,page_239.md,"[-0.02881, 0.02509, -0.05682, 0.01467, 0.00049..."


In [18]:
dataframe["folder"][0]

'/content/drive/MyDrive/006 html/document_1_html/clean_html/clean_markdown'

In [19]:
# Вытаскиваем номер каталога
dataframe["folder"] = dataframe["folder"].apply(lambda x: x.split("_")[1]).astype(int)
dataframe

,folder,file_name,embedding
0,1,page_86.md,"[-0.03644, 0.004272, -0.0418, -0.007412, 0.016..."
1,1,page_272.md,"[-0.02133, 0.01595, -0.055, -0.03708, -0.00193..."
2,1,page_288.md,"[-0.002651, 0.01232, -0.04044, -0.05136, 0.007..."
3,1,page_37.md,"[-0.02913, 0.02794, -0.0411, -0.03366, 0.01168..."
4,1,page_239.md,"[-0.02881, 0.02509, -0.05682, 0.01467, 0.00049..."
...,...,...,...
607,7,page_22.md,"[0.002947, -0.006577, -0.01381, 0.003998, -0.0..."
608,7,page_10.md,"[-0.0525, -0.011246, -0.00761, 0.03198, -0.009..."
609,7,page_18.md,"[-0.01312, -0.001345, -0.0321, 0.00526, 0.0067..."
610,7,page_19.md,"[-0.04056, 0.01087, -0.0461, 0.001358, 0.00061..."


In [20]:
# Вытаскиваем номер страницы
dataframe["file_name"] = dataframe["file_name"].apply(lambda x: x.split("_")[1]).apply(lambda y: y.split(".")[0]).astype(int)
dataframe

,folder,file_name,embedding
0,1,86,"[-0.03644, 0.004272, -0.0418, -0.007412, 0.016..."
1,1,272,"[-0.02133, 0.01595, -0.055, -0.03708, -0.00193..."
2,1,288,"[-0.002651, 0.01232, -0.04044, -0.05136, 0.007..."
3,1,37,"[-0.02913, 0.02794, -0.0411, -0.03366, 0.01168..."
4,1,239,"[-0.02881, 0.02509, -0.05682, 0.01467, 0.00049..."
...,...,...,...
607,7,22,"[0.002947, -0.006577, -0.01381, 0.003998, -0.0..."
608,7,10,"[-0.0525, -0.011246, -0.00761, 0.03198, -0.009..."
609,7,18,"[-0.01312, -0.001345, -0.0321, 0.00526, 0.0067..."
610,7,19,"[-0.04056, 0.01087, -0.0461, 0.001358, 0.00061..."


In [21]:
# Делаем матрицу эмбеддингов
matrix_embedding = np.row_stack(dataframe["embedding"].to_list())
matrix_embedding

<ipython-input-21-534b6a04d819>:2: DeprecationWarning: `row_stack` alias is deprecated. Use `np.vstack` directly.
  matrix_embedding = np.row_stack(dataframe["embedding"].to_list())


array([[-0.03644 ,  0.004272, -0.0418  , ..., -0.00948 ,  0.05453 ,
         0.0207  ],
       [-0.02133 ,  0.01595 , -0.055   , ...,  0.01369 ,  0.0351  ,
         0.05084 ],
       [-0.002651,  0.01232 , -0.04044 , ...,  0.011314,  0.0324  ,
         0.05588 ],
       ...,
       [-0.01312 , -0.001345, -0.0321  , ..., -0.004807, -0.0468  ,
        -0.03232 ],
       [-0.04056 ,  0.01087 , -0.0461  , ..., -0.016   , -0.064   ,
        -0.015564],
       [-0.05725 , -0.0031  , -0.03604 , ..., -0.02982 ,  0.0808  ,
        -0.01163 ]], dtype=float16)

In [22]:
# Проверяем вектора на евклидову норму
np.linalg.norm(matrix_embedding[0])

np.float16(1.0)

In [23]:
# Проверяем размерность матрицы
matrix_embedding.shape

(612, 1024)

In [24]:
# Записываем в csv колонки с папкой и номером страницы
dataframe[["folder", "file_name"]].to_csv("catalog_page_md.csv")

In [25]:
dim = 1024 # размерность эмбеддинга

In [26]:
# Создаем faiss индекс
index_md = faiss.IndexFlatIP(dim) # IP - внутренний продукт (скалярное произведение)

In [27]:
# в индекс добавляем наши вектора
index_md.add(matrix_embedding)

In [29]:
# Проверяем количество веторов в индексе
index_md.ntotal

612

In [89]:
query = "Какие кабели используются для пожароопасных помещений?"

In [90]:
query_embedding_md = model.encode(query, batch_size = 1, max_length = 512)['dense_vecs']

In [91]:
# D - косинусное сходство, I - индексы
D,I = index_md.search(query_embedding_md.reshape((1, 1024)), 10)

In [92]:
D,I

(array([[0.70142394, 0.6995252 , 0.6959717 , 0.69361883, 0.69079053,
         0.68041605, 0.6787898 , 0.6778754 , 0.67630523, 0.6708062 ]],
       dtype=float32),
 array([[172,   4, 258,  14, 133, 105, 320,  19,  38, 184]]))

In [93]:
# сохраняем индекс в файл
faiss.write_index(index_md, "index_md.faiss")

In [94]:
# чтение индекса из файла
index_md = faiss.read_index("index_md.faiss")

In [95]:
dataframe_new = pd.read_csv("catalog_page_md.csv", index_col = 0)
dataframe_new

,folder,file_name
0,1,86
1,1,272
2,1,288
3,1,37
4,1,239
...,...,...
607,7,22
608,7,10
609,7,18
610,7,19


In [96]:
# по индексам из faiss вытаскиваем сами страницы в строковом виде
def list_texts(df, list_index, path):
    selected_rows = df.iloc[list_index, :]
    print(selected_rows)
    list_text = []
    for index, row in selected_rows.iterrows():
        folder = row["folder"]
        folder_name = f"document_{folder}_html"
        page = row["file_name"]
        page_name = f"page_{page}.md"
        full_name = os.path.join(path, folder_name, "clean_html", "clean_markdown", page_name)
        with open (full_name) as f:
            html = f.read()
            list_text.append(html)
    return list_text

In [98]:
# список со всеми текстами, подходящими по запросу
texts = list_texts(dataframe_new, [172,   4, 258,  14, 133, 105, 320,  19,  38, 184], "/content/drive/MyDrive/006 html")
texts

     folder  file_name
172       1        235
4         1        239
258       1        224
14        1        230
133       1        237
105       1        218
320       2        117
19        1        219
38        1        195
184       1        222


['Стр. 244\n\n**КОМАНДЫ**\n\n* **КАБЕЛИ ДЛЯ СИСТЕМ ПОЖАРНОЙ СИГНАЛИЗАЦИИ**\n  \n  * **Предназначены для групповой стационарной прокладки**\n  * **в системах противопожарной защиты, охранно-пожарной сигнализации, оповещения и управления эвакуацией людей**\n  * **при пожаре**, в зданиях, сооружениях и строениях (промышленных предприятиях, школах, больницах, офисных помещениях, высотных зданиях, метро, атомных станциях), \n  * **где предъявляются требования к кабелям и проводам по сохранению работоспособности в условиях пожара. Сохраняют работуоспособность при пожаре в течение 240 минут .**\n  \n  * **Эксплуатируется внутри и вне помещений, при условии защиты от прямого воздействия солнечного излучения и атmosферных осадков.**\n  \n  * **Класс пожарной опасности по ГОСТ 31565 P1б.1.1.2.1.**\n  \n  * **Конструкция**\n    \n    * **Токопроводящая жила:** меди́нная, однопроволочная, I класса или многопроволочная не ниже III класса по ГОСТ 22483, сечением от 0,2; 0,35; 0,5; 0,75; 1,0; 1,5; 2,

In [99]:
# функция для создания нового промта из текста запроса и текста документа
def generate_promt_3(query, list_texts):
    prefix = "Запрос на получение информации."
    prefix += f"\nЗапрос: {query}\n"
    for text in list_texts:
        prefix += f"Документ:\n{text}\n"
    prefix += "На основе представленных документов и запроса пользователя сформулируй ответ.\nУчти, что найденные документы могут не отвечать на запрос. Если список документов пустой, то ответь что по запросу ничего не найдено. Представь ответ в виде обычного текста."
    return prefix

In [100]:
prompt = generate_promt_3(query, texts)
prompt

'Запрос на получение информации.\nЗапрос: Какие кабели используются для пожароопасных помещений?\nДокумент:\nСтр. 244\n\n**КОМАНДЫ**\n\n* **КАБЕЛИ ДЛЯ СИСТЕМ ПОЖАРНОЙ СИГНАЛИЗАЦИИ**\n  \n  * **Предназначены для групповой стационарной прокладки**\n  * **в системах противопожарной защиты, охранно-пожарной сигнализации, оповещения и управления эвакуацией людей**\n  * **при пожаре**, в зданиях, сооружениях и строениях (промышленных предприятиях, школах, больницах, офисных помещениях, высотных зданиях, метро, атомных станциях), \n  * **где предъявляются требования к кабелям и проводам по сохранению работоспособности в условиях пожара. Сохраняют работуоспособность при пожаре в течение 240 минут .**\n  \n  * **Эксплуатируется внутри и вне помещений, при условии защиты от прямого воздействия солнечного излучения и атmosферных осадков.**\n  \n  * **Класс пожарной опасности по ГОСТ 31565 P1б.1.1.2.1.**\n  \n  * **Конструкция**\n    \n    * **Токопроводящая жила:** меди́нная, однопроволочная, I к

In [102]:
# сам запрос
response = send_request_to_vllm(api_url, prompt, model_name)
print(response)

Исходя из предоставленных документов, основные кабели, используемые для пожароопасных помещений, это:

1. **Кабели огнестойкие, низкотоксичные, не распространяющие горение при групповой прокладке, с пониженным дымовым и газовым выделением (исполнение – "нг-FRLSL Tx").**
   - Это кабели, которые соответствуют требованиям ГОСТ 31565 П1б.1.2.2.2 и имеют класс пожарной опасности по ГОСТ 31565 P1б.1.2.1.2.
   - Они предназначены для групповой стационарной прокладки в системах противопожарной защиты, охранно-пожарной сигнализации, оповещения и управления эвакуацией людей при пожаре.
   - Эти кабели имеют термический барьер, изолированную внутреннюю оболочку и броню, а также снижают светопроницаемость до 50%.

2. **Кабели огнестойкие, низкотоксичные, не распространяющие горение при групповой прокладке (исполнение – "нг-FRLS").**
   - Эти кабели также соответствуют требованиям ГОСТ 31565 П1б.1.2.2.2 и имеют класс пожарной опасности по ГОСТ 31565 P1б.1.2.1.2.
   - Они предназначены для группово